# Analysis of the Temporal Evolution of Loads in the WECC

This notebook analyzes the time series of annual total and peak loads from TELL for the BAs in the WECC.

In [9]:
# Start by importing the packages we need:
import os
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Suppress Future Warnings


In [10]:
warnings.simplefilter(action='ignore', category=FutureWarning)


## Set the Directory Structure

In [2]:
# Identify the top-level directory and the subdirectory where the data will be stored:
load_data_input_dir =  '/Users/burl878/Documents/GODEEEP/Data/Merged_BA_Loads/'
data_output_dir =  '/Users/burl878/Documents/Code/code_repos/load_analysis/data/'
image_output_dir =  '/Users/burl878/Documents/Code/code_repos/load_analysis/figures/'


## Process the GridView-Ready Output Files

In [22]:
# Check to see if the processed output file exist and if not then create it:
if os.path.isfile((os.path.join(data_output_dir, 'WECC_Load_Time_Series.csv'))) == False:

   # Set the scenarios to process:
   scenarios = ['bau_climate', 'bau_ira_ccs_climate', 'nz_climate', 'nz_ccs_climate', 'nz_ira_ccs_climate']
    
   #Initiate a counter and empty dataframe to store the results:
   counter = 0;
   output_df = pd.DataFrame()

   # Loop over the scenarios and process each time series:
   for s in range(len(scenarios)):
       # Set the years to process:
       years_to_process = [2025, 2030, 2035, 2040, 2045, 2050]
              
       # Loop over the years from the start_year to the end_year:
       for year in years_to_process:
           # Read in the TELL BA output file for that year and scenario:
           gv_df = pd.read_csv(load_data_input_dir + 'GridView_Loads_' + scenarios[s] + '_' + str(year) + '.csv')
   
           # Strip the unecessary bits from the column names:
           gv_df.columns = gv_df.columns.str.replace("_CEC", "")
           gv_df.columns = gv_df.columns.str.replace("_2030.dat", "")
           gv_df.columns = gv_df.columns.str.replace("Load_", "")
    
           # Delete the random column:
           # del gv_df["Unnamed: 38"]
    
           # Drop the first row and last 4 rows:
           gv_df = gv_df.tail(-1).iloc[:-4, :]
    
           # Compute the hourly total load across BAs:
           gv_df['GV_Total_Load_MWh'] = gv_df.sum(axis=1)
    
           # Copy the total load to a new dataframe:
           gv_load_df = gv_df[['Index', 'GV_Total_Load_MWh']].copy().iloc[0:8760, :]
           gv_load_df.rename(columns={'Index': 'Hour'}, inplace=True)
           
           # Iterate the counter by one:
           counter = counter + 1
            
           # Put the output in a new dataframe:
           output_df.loc[counter, 'Scenario'] = scenarios[s]
           output_df.loc[counter, 'Year'] = int(year)
           output_df.loc[counter, 'Total_Load_TWh'] = (gv_load_df['GV_Total_Load_MWh'].sum().round(6)) / 1000000
           output_df.loc[counter, 'Peak_Load_MWh'] = gv_load_df['GV_Total_Load_MWh'].max()
            
           # Clean up
           del gv_df, gv_load_df
         
   # Write out the dataframe to a .csv file:
   output_df.to_csv((os.path.join(data_output_dir, 'WECC_Load_Time_Series.csv')), sep=',', index=False)
   
else:
   # Read in the already processed output file:
   output_df = pd.read_csv((os.path.join(data_output_dir, 'WECC_Load_Time_Series.csv')))            
                  
# Preview the future dataframe:
output_df


,Scenario,Year,Total_Load_TWh,Peak_Load_MWh
0,bau_climate,2025.0,1062.549144,191238.22
1,bau_climate,2030.0,1183.366206,209899.52
2,bau_climate,2035.0,1262.619787,224834.52
3,bau_climate,2040.0,1354.529614,238809.18
4,bau_climate,2045.0,1438.640575,254611.33
5,bau_climate,2050.0,1524.692093,263914.67
6,bau_ira_ccs_climate,2025.0,1060.753001,189831.94
7,bau_ira_ccs_climate,2030.0,1208.736478,214699.80
8,bau_ira_ccs_climate,2035.0,1325.829548,236943.40
9,bau_ira_ccs_climate,2040.0,1372.755392,242189.98


In [45]:
# Define a function to plot the time series of annual demand for the WECC:
def plot_wecc_load_time_series(data_input_dir: str, image_output_dir: str, image_resolution: int, save_images=False):
    
    # Read in the output file:
    output_df = pd.read_csv((os.path.join(data_output_dir, 'WECC_Load_Time_Series.csv')))
    
    # Subset to the time series for each interconnection and scenario:
    wecc_bau_climate = output_df[output_df['Scenario'].isin(['bau_climate'])]
    wecc_bau_ira_ccs_climate = output_df[output_df['Scenario'].isin(['bau_ira_ccs_climate'])]
    wecc_nz_climate = output_df[output_df['Scenario'].isin(['nz_climate'])]
    wecc_nz_ccs_climate = output_df[output_df['Scenario'].isin(['nz_ccs_climate'])]    
    wecc_nz_ira_ccs_climate = output_df[output_df['Scenario'].isin(['nz_ira_ccs_climate'])]     
    
    # Make the raw load plots:
    plt.figure(figsize=(24, 12))
    plt.rcParams['font.size'] = 18
    
    plt.subplot(211)
    plt.plot(wecc_bau_climate['Year'], wecc_bau_climate['Total_Load_TWh'], color='k', linestyle='-', label='bau_climate', linewidth=3)
    plt.plot(wecc_bau_ira_ccs_climate['Year'], wecc_bau_ira_ccs_climate['Total_Load_TWh'], color='b', linestyle='-', label='bau_ira_ccs_climate', linewidth=3)
    plt.plot(wecc_nz_climate['Year'], wecc_nz_climate['Total_Load_TWh'], color='r', linestyle='-', label='nz_climate', linewidth=3)
    plt.plot(wecc_nz_ccs_climate['Year'], wecc_nz_ccs_climate['Total_Load_TWh'], color='g', linestyle='-', label='nz_ccs_climate', linewidth=3)
    plt.plot(wecc_nz_ira_ccs_climate['Year'], wecc_nz_ira_ccs_climate['Total_Load_TWh'], color='m', linestyle='-', label='nz_ira_ccs_climate', linewidth=3)
    plt.xlim([2025, 2050]); plt.xticks([2025, 2030, 2035, 2040, 2045, 2050],['2025','2030','2035','2040','2045','2050'])
    plt.ylabel('Annual Total Load [TWh]')
    plt.grid(True)
    plt.legend(loc='upper left', prop={'size': 18})
    plt.title('Western Interconnection Annual Energy Projections')
    plt.title('a)', loc='left', fontsize=15)
    
    plt.subplot(212)
    plt.plot(wecc_bau_climate['Year'], wecc_bau_climate['Peak_Load_MWh'], color='k', linestyle='-', label='bau_climate', linewidth=3)
    plt.plot(wecc_bau_ira_ccs_climate['Year'], wecc_bau_ira_ccs_climate['Peak_Load_MWh'], color='b', linestyle='-', label='bau_ira_ccs_climate', linewidth=3)
    plt.plot(wecc_nz_climate['Year'], wecc_nz_climate['Peak_Load_MWh'], color='r', linestyle='-', label='nz_climate', linewidth=3)
    plt.plot(wecc_nz_ccs_climate['Year'], wecc_nz_ccs_climate['Peak_Load_MWh'], color='g', linestyle='-', label='nz_ccs_climate', linewidth=3)
    plt.plot(wecc_nz_ira_ccs_climate['Year'], wecc_nz_ira_ccs_climate['Peak_Load_MWh'], color='m', linestyle='-', label='nz_ira_ccs_climate', linewidth=3)
    plt.xlim([2025, 2050]); plt.xticks([2025, 2030, 2035, 2040, 2045, 2050],['2025','2030','2035','2040','2045','2050'])
    plt.ylabel('Annual Peak Demand [MW]')
    plt.grid(True)
    plt.title('Western Interconnection Peak Demand Projections')
    plt.title('b)', loc='left', fontsize=15)
    
    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       filename = ('WECC_Load_Projections.png')
       plt.savefig(os.path.join(image_output_dir, filename), dpi=image_resolution, bbox_inches='tight', facecolor='white')
       plt.close()
    

In [47]:
plot_wecc_load_time_series(data_input_dir = data_output_dir, 
                           image_output_dir = image_output_dir, 
                           image_resolution = 300, 
                           save_images = True)
